In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers.legacy import Adam

from art import config
from art.utils import load_dataset, get_file
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, ProjectedGradientDescent
from art.defences.trainer import AdversarialTrainer

def BuiltModel(bArt):
    (x_train, y_train), (x_test, y_test), min_, max_ = load_dataset('mnist')
    print(x_train.shape, y_train.shape)

    aModel = Sequential()
    aModel.add(Conv2D(filters=32, kernel_size=(3, 3), 
                                strides=1, activation="relu", 
                                input_shape=(28, 28, 1)))
    aModel.add(MaxPooling2D(pool_size=(2, 2)))
    aModel.add(Conv2D(filters=64, kernel_size=(3, 3), 
                                strides=1, activation="relu", 
                                input_shape=(23, 23, 4)))
    aModel.add(MaxPooling2D(pool_size=(2, 2)))
    aModel.add(Flatten())
    aModel.add(Dense(128, activation="relu"))
    aModel.add(Dense(10, activation="softmax"))

    aModel.compile(loss=categorical_crossentropy, 
                            optimizer=Adam(learning_rate=1e-4), 
                            metrics=["accuracy"])

    aModel.summary()

    if bArt:
        classifier = KerasClassifier(clip_values=(min_, max_), model=aModel, use_logits=False)
        classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128, verbose=1)
        #classifier.model.save("./mnist_cnn_original.h5")
    else:
        aBatchSz = 128        
        aEpochs = 1
        hist = aModel.fit(x_train, y_train, 
                  batch_size=aBatchSz, 
                  epochs=aEpochs, 
                  validation_split=0.1, 
                  verbose=1)
        
        losses = pd.DataFrame(aModel.history.history)
        losses[['loss','val_loss']].plot()
        losses[['accuracy','val_accuracy']].plot()


In [ ]:
BuiltModel(False)